In [ ]:
import ee
import geepy
import math
import pandas as pd

ee.Initialize()

In [ ]:
config = geepy.params.configParams('input.json')

amostras = ee.FeatureCollection(config.params['samples'])

watersheds = ee.FeatureCollection(config.params['studyArea'])

bands = config.params['bandParams']

l5 = ee.ImageCollection(config.params['imgCollection']['lc5']['id']).select(
    config.params["imgCollection"]["lc5"]["bands"],
    config.params["imgCollection"]["lc5"]["bandNames"])

l7 = ee.ImageCollection(config.params['imgCollection']['lc7']['id']).select(
    config.params["imgCollection"]["lc7"]["bands"],
    config.params["imgCollection"]["lc7"]["bandNames"])

l8 = ee.ImageCollection(config.params['imgCollection']['lc8']['id']).select(
    config.params["imgCollection"]["lc8"]["bands"],
    config.params["imgCollection"]["lc8"]["bandNames"])

In [ ]:
landsat = {}

for year in config.params['years2process']:
    start_d = year + config.params['period']['dry']['start']
    end_d = year + config.params['period']['dry']['end']
    #print("Start-End dry season %s %s" %(start_d, end_d))

    if(int(year) < 2002):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d)
    elif(int(year) in (2002,2011,2012)):
        filtered = l7.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d)
    elif(int(year) > 2002 and int(year) < 2011):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d)
    else:
        filtered = l8.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d)

    fEdgeRemoved = filtered.map(geepy.image.edgeRemoval).median()
    
    landsat[year] = fEdgeRemoved.clip(watersheds)

In [ ]:
landsat['2016'] = geepy.image.tassCapTransformation(landsat['2016'], 'l8')

In [ ]:
landsat['2016'] = geepy.image.calcBCI(landsat['2016'], watersheds)

In [ ]:
mapa = geepy.maps.geeMap(watersheds, zoom=8)

viz_params = {'min':0,'max':0.5,'bands':'swir1,nir,red','format':'png'}

viz_tasscap = {'min':-0.3,'max':1}

mapa.addLayer(landsat['2016'].select('bci'), viz_params=viz_tasscap, name='landsat2016')
# mapa.addLayer(tc.select('brightness'), name ='g')
# mapa.addLayer(tc.select('greenness'), name ='b')
# mapa.addLayer(tc.select('wetness'), name ='w')

mapa.addControls()